In [39]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import requests
import os

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
print(f"alpha_vantage_api_key: {alpha_vantage_api_key}")

def get_symbol(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    result = ddg.run(f"Ticker Symbol for {company_name}")
    return result

def get_income_statement(inputs):
    symbol = inputs["symbol"]
    print(symbol)
    print(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
    r = requests.get(
        f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
    return r.json()["annualReports"]

def get_balance_sheet(inputs):
    symbol = inputs["symbol"]
    r = requests.get(
        f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}")
    return r.json()["annualReports"]

def get_daily_stock_performance(inputs):
    symbol = inputs["symbol"]

    r = requests.get(
        f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
    response = r.json()
    return list(response["Weekly Time Series"].keys())[:200]

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_symbol",
            "description": "Given the name of a company returns its stock market symbol. Example: AAPL, TSLA, GOOGL, etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a stock market symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "the stock market symbol of the company",
                    },
                },
                "required": ["symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a stock marketsymbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "the stock market symbol of the company",
                    },
                },
                "required": ["symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a symbol symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "the stock market symbol of the company",
                    },
                },
                "required": ["symbol"],
            },
        },
    },
]
get_income_statement({"symbol": "NET"})

alpha_vantage_api_key: W6C3T0TW6HC0TLY5
NET
https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=NET&apikey=W6C3T0TW6HC0TLY5


KeyError: 'annualReports'

In [36]:
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not.",
    model="gpt-3.5-turbo-1106",
    tools=functions,
)

# Create a thread
thread = client.beta.threads.create()

# Add a message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if the Cloudflare stock is a good buy?"
)

# Run the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Wait for the run to complete
while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run_status.status == 'completed':
        break
    elif run_status.status == 'failed':
        raise Exception("Run failed")
    import time
    time.sleep(1)

# Get the messages
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Print the assistant's response
for message in messages.data:
    if message.role == "assistant":
        print(message.content[0].text.value)

KeyboardInterrupt: 